In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm, tqdm_pandas
import matplotlib.pyplot as plt

tqdm.pandas()

url = 'https://ballershoesdb.com/players/'
page = requests.get(url)
html_contents = page.text
soup = BeautifulSoup(html_contents, 'html.parser')

In [7]:
urls = ['https://ballershoesdb.com/players/']
for i in range(1, 40):
    print(i, end=" ")
    url = f"https://ballershoesdb.com/players/page/{i}/"
    page = requests.get(url)
    html_contents = page.text
    soup = BeautifulSoup(html_contents, 'html.parser')
    if len(soup.find_all("div", class_='item-inside')):
        urls.append(url)
    else:
        break

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 

In [8]:
def get_soup(url):
    page = requests.get(url)
    html_contents = page.text
    soup = BeautifulSoup(html_contents, 'html.parser')
    return soup
    
players = []
for i, url in enumerate(urls):
    print(i, end=" ")
    soup = get_soup(url)
    player_cards = soup.find_all('div', class_='item-inside')
    for player in player_cards:
        player_page = player.find_all('a', href=True)[0]['href']
        players.append((player.find('h3').text, player_page))

print()
print(f'found {len(players)} player pages')
print(*players[:10], sep="\n")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 
found 446 player pages
('Precious Achiuwa', 'https://ballershoesdb.com/players/precious-achiuwa/')
('Steven Adams', 'https://ballershoesdb.com/players/steven-adams/')
('Bam Adebayo', 'https://ballershoesdb.com/players/bam-adebayo/')
('Santi Aldama', 'https://ballershoesdb.com/players/santi-aldama/')
('Nickeil Alexander-Walker', 'https://ballershoesdb.com/players/nickeil-alexander-walker/')
('Grayson Allen', 'https://ballershoesdb.com/players/grayson-allen/')
('Jarrett Allen', 'https://ballershoesdb.com/players/jarrett-allen/')
('Jose Alvarado', 'https://ballershoesdb.com/players/jose-alvarado/')
('Kyle Anderson', 'https://ballershoesdb.com/players/kyle-anderson/')
('Giannis Antetokounmpo', 'https://ballershoesdb.com/players/giannis-antetokounmpo/')


In [10]:
def scrape_kicks(url):
#     try:
    page = requests.get(url)
    html_contents = page.text
    soup = BeautifulSoup(html_contents, 'html.parser')
    
    # scrape current shoes
    current_shoes = soup.find("div", class_="first post-list-item")
    if current_shoes is None:
        kicks = ''
    else:
        kicks = current_shoes.find('h3').text.strip()

    position_div = soup.find("div", class_="post-title")
    position = position_div.find("h2").findAll("span")[1].getText()
    return kicks, position
    
test_url = "https://ballershoesdb.com/players/santi-aldama/"
scrape_kicks(test_url)

('Nike Kyrie Low 5', 'Forward')

In [11]:
df = pd.DataFrame(columns=['Player', 'Player Page Link'], data=players)
df[['Current Kicks', 'Position']] = df['Player Page Link'].progress_apply(lambda x: scrape_kicks(x)).to_list()
df

100%|██████████| 446/446 [04:14<00:00,  1.75it/s]


,Player,Player Page Link,Current Kicks,Position
0,Precious Achiuwa,https://ballershoesdb.com/players/precious-ach...,Anta Daringly 3.0,Forward
1,Steven Adams,https://ballershoesdb.com/players/steven-adams/,New Balance Two Wxy V3,Center
2,Bam Adebayo,https://ballershoesdb.com/players/bam-adebayo/,Air Jordan 37,Forward
3,Santi Aldama,https://ballershoesdb.com/players/santi-aldama/,Nike Kyrie Low 5,Forward
4,Nickeil Alexander-Walker,https://ballershoesdb.com/players/nickeil-alex...,Nike Kobe 5 Protro,Guard
...,...,...,...,...
441,Thaddeus Young,https://ballershoesdb.com/players/thaddeus-young/,Nike Kobe 6 Protro,Forward
442,Trae Young,https://ballershoesdb.com/players/trae-young/,Adidas Trae Young 2,Guard
443,Omer Yurtseven,https://ballershoesdb.com/players/omer-yurtseven/,Nike Kobe 5 Protro,Center
444,Cody Zeller,https://ballershoesdb.com/players/cody-zeller/,Air Jordan 37,Center


In [12]:
df.to_csv('player-shoes-data.csv', index=False)

In [20]:
kicks_df = df.groupby(['Current Kicks', 'Position'])['Player'].count().reset_index()
len(kicks_df['Current Kicks'].unique())

136